In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [42]:
torch.manual_seed(42)

In [43]:
df =pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [44]:
df.shape

(6000, 785)

In [45]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values


In [46]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [47]:
x_train=x_train/255.0
x_test=x_test/255.0

In [48]:
x_train

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.0627451, 0.       ,
        0.       ]])

In [49]:
#create customeDataset Class
class CustomDataset(Dataset):
    def __init__(self,features,labels):
      self.features=torch.tensor(features,dtype=torch.float32)
      self.labels=torch.tensor(labels,dtype=torch.long)

    def  __len__(self):
      return len(self.features)

    def __getitem__(self,item):
      return self.features[item],self.labels[item]



In [50]:
# Train dataset object create
train_dataset=CustomDataset(x_train,y_train)
len(train_dataset)


4800

In [51]:
test_dataset=CustomDataset(x_test,y_test)
len(test_dataset)

1200

In [52]:
# train_test_loader create
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [53]:
len(train_loader)

150

In [54]:
class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )

  def forward(self, x):  # Properly aligned at class level
    return self.model(x)


In [55]:
epochs=100
learning_rate=0.1

In [56]:
#instantiate the model
model=MyNN(x_train.shape[1]) #show data in features

#loss function
criterion=nn.CrossEntropyLoss()

#optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate)



In [57]:
# training_loop
for epoch in range(epochs):
    total_epoch_loss=0
    for batch_features, batch_labels in train_loader:
        # forward pass
        outputs = model(batch_features)

        # calculate loss
        loss = criterion(outputs, batch_labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()

        # update gradients
        optimizer.step()
        total_epoch_loss=total_epoch_loss+loss.item()
    avg_loss=total_epoch_loss/len(train_loader)
    print(f'epoch:{epoch+1}, loss: {avg_loss}')

epoch:1, loss: 1.3216368440786999
epoch:2, loss: 0.7793365436792373
epoch:3, loss: 0.6427524652083715
epoch:4, loss: 0.5751657336950302
epoch:5, loss: 0.528180201848348
epoch:6, loss: 0.49528750121593473
epoch:7, loss: 0.4602432996034622
epoch:8, loss: 0.4359459094206492
epoch:9, loss: 0.41829214423894884
epoch:10, loss: 0.3984092291692893
epoch:11, loss: 0.3850264506538709
epoch:12, loss: 0.3722685620188713
epoch:13, loss: 0.3477362078428268
epoch:14, loss: 0.3467242218057315
epoch:15, loss: 0.3138812760512034
epoch:16, loss: 0.3126461291313171
epoch:17, loss: 0.29552092105150224
epoch:18, loss: 0.2857150869568189
epoch:19, loss: 0.27084775785605114
epoch:20, loss: 0.26237100958824155
epoch:21, loss: 0.25756089876095456
epoch:22, loss: 0.24232648963729542
epoch:23, loss: 0.23957946779827277
epoch:24, loss: 0.2261324493587017
epoch:25, loss: 0.2233684351791938
epoch:26, loss: 0.21165308925012746
epoch:27, loss: 0.21910773759086927
epoch:28, loss: 0.21103892110288144
epoch:29, loss: 0.1

In [58]:
# model evaluation perpose
model.eval()


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [61]:
# eval code
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    outputs=model(batch_features)

    _,predicted=torch.max(outputs,1)

    total=total+batch_labels.shape[0]
    correct=correct+(predicted==batch_labels).sum().item()
print(correct/total)

0.8325


In [ ]:
# improve accuracy value
use full dataset(large date set use gpu)
optimizer,lr,epochs,weight_int
hpt(hyper parameter training)